In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
from numpy import dot
from numpy.linalg import norm

In [ ]:

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import cv2
import gc

In [ ]:
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
from gensim.models import Word2Vec
import nltk
# nltk.download('wordnet')
stemmer = SnowballStemmer('english')

from numpy import dot
from numpy.linalg import norm

In [ ]:
train_df = pd.read_csv('../input/shopee-product-matching/train.csv')
# train_df

In [ ]:
test_df = pd.read_csv('../input/shopee-product-matching/test.csv')
# test_df

In [ ]:
train_title_list = train_df['title'].tolist()
train_label_list = train_df['label_group'].tolist()
# train_title_list
# train_label_list

In [ ]:
test_title_list = test_df['title'].tolist()
test_title_list

In [ ]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            if token == 'xxxx':
                continue
            result.append(lemmatize_stemming(token))
    
    return result

# Process title field for train df

In [ ]:
processed_docs = train_df['title'].map(preprocess)
processed_docs =list(processed_docs)

In [ ]:
def word2vec_model():
    w2v_model = Word2Vec(min_count=1,
                     window=3,
                     vector_size=50,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20)
    
    w2v_model.build_vocab(processed_docs)
    w2v_model.train(processed_docs, total_examples=w2v_model.corpus_count, epochs=300, report_delay=1)
    
    return w2v_model

In [ ]:
w2v_model = word2vec_model()

# Getting embedding vector

In [ ]:
emb_vec = w2v_model.wv
model = emb_vec

# Finding similarity between two vector using cosine similarity

In [ ]:
def find_similarity(sen1, sen2, model):
    p_sen1 = preprocess(sen1)
    p_sen2 = preprocess(sen2)
    
    sen_vec1 = np.zeros(50)
    sen_vec2 = np.zeros(50)
    for val in p_sen1:
        sen_vec1 = np.add(sen_vec1, model[val])
#     print("sen_vec1 ", sen_vec1)
    for val in p_sen2:
        if val in model.key_to_index:
            sen_vec2 = np.add(sen_vec2, model[val])
#     print("sen_vec2 ", sen_vec2)
    return dot(sen_vec1,sen_vec2)/(norm(sen_vec1)*norm(sen_vec2))

# Some optimization

In [ ]:
sen_vec1_arr = []
for train_data in train_title_list:
    p_sen1 = preprocess(train_data)
    sen_vec1 = np.zeros(50)
    for val in p_sen1:
        sen_vec1 = np.add(sen_vec1, model[val])
    sen_vec1_arr.append(sen_vec1)

In [ ]:
from datetime import datetime
start_time = datetime.now()
pred_df = []
pcounter = 0
model = emb_vec
    
for test_data in test_title_list:
    sim_score = {}
    p_sen2 = preprocess(test_data)
    sen_vec2 = np.zeros(50)
    for val in p_sen2:
        if val in model.key_to_index:
            sen_vec2 = np.add(sen_vec2, model[val])
    index = 0
    for train_data in train_title_list:
        sen_vec1 = sen_vec1_arr[index]
        distance = dot(sen_vec1,sen_vec2)/(norm(sen_vec1)*norm(sen_vec2))
#         if distance > 0.6:
        sim_score[train_data] = distance
        index = index+1
    
    sorted_dict = {k: v for k, v in sorted(sim_score.items(), key=lambda item: item[1], reverse=True)}

    first = list(sorted_dict.keys())[0]
#     sim_score = next(iter(sorted_dict.values()))
#     print("pcounter ", pcounter)
#     print("sim_score key ", first)
#     print("sim_score score ", sim_score)    
#     print("test_data ", test_data)
    index = train_title_list.index(first)
#     print("index ", index)
#     print("train_title_list[index] ", train_label_list[index])
    pred_df.append(train_label_list[index])
    pcounter = pcounter+1
end_time = datetime.now()
print('Program took : {}'.format(end_time - start_time))

In [ ]:
# pred_df = []
# for test_data in test_title_list:
#     sim_score = {}
#     for train_data in train_title_list:
#         sim_score[train_data] = find_similarity(train_data,test_data,emb_vec)
    
#     sorted_dict = {k: v for k, v in sorted(sim_score.items(), key=lambda item: item[1], reverse=True)}
    
#     first = next(iter(sorted_dict.keys()))
# #     print("sim_score ", first)
# #     print("test_data ", test_data)
#     index = train_title_list.index(first)
# #     print("index ", index)
# #     print("train_title_list[index] ", train_label_list[index])
#     pred_df.append(train_label_list[index])

In [ ]:
pred_df

In [ ]:
submission_df = test_df

In [ ]:
submission_df['pred'] = [element for element in pred_df]

In [ ]:
submission_df

In [ ]:
drop_cols = ['image','image_phash','title']
submission_df = submission_df.drop(columns=drop_cols)

In [ ]:
submission_df['matches'] = submission_df.groupby(['pred'])['posting_id'].transform(lambda x : ' '.join(x))


In [ ]:
submission_df = submission_df.drop(columns='pred')
submission_df

In [ ]:
submission_df.to_csv('submission.csv', index = False)